<h1>Wild Card</h1>

<h5>Bigram</h5>

In [ ]:
def generateBigram(_queri):
    hasil = []
    for i in range(len(_queri)-1):
        if i == 0:
            hasil.append('$'+_queri[i])
        else:
            hasil.append(_queri[i] + _queri[i+1])
            
    return hasil

In [ ]:
def wildCard(_queri):
    queri = _queri.replace('*','')
    bigram = generateBigram(queri)
    
    return bigram

<h1>Koreksi Ejaan</h1>

<h5>Kata Terisolasi</h5>

In [1]:
from nltk.corpus import words #kamus inggris
from nltk.metrics import * #hitung jarak

In [9]:
def spellingCorection(_word):
    wordsSelected = identicalWord(_word)
    
    thresholdDistance = 3 #threshold
    mostIdenticalWord = []
    for i in wordsSelected: #check dengan levenshtaien
        tempDistance = edit_distance(i, _word)
        if tempDistance < thresholdDistance:
            sortestDistance = tempDistance
            mostIdenticalWord.append(i)
            
    return mostIdenticalWord

In [10]:
def identicalWord(_word):
    wordLenght = len(_word)
    wordsSelected = [] #word yang selisih panjang = 0
    
    for word in words.words():
        if abs(len(word) - wordLenght) == 0 and word[0] == _word[0]: #panjangnya sama dan huruf depan sama
            wordsSelected.append(word)
            
    return wordsSelected

In [11]:
a = spellingCorection('heigth')
print(a)

['health', 'hearth', 'height']


<h1>Sounddex</h1>

In [ ]:
def makeSounddex(_listOfTerm):
    